In [147]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3 as sql
from datetime import timedelta, datetime
#### Plotly ####
import ipywidgets
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

#### Import Data ####
conn = sql.connect('C:/Users/Simplon/Desktop/PROJET_FINAL_BEAUTY/App_Beauty_E1/Application_prod.db')
data = pd.read_sql("""SELECT * 
                      FROM Raw_data""", conn)
predict_data = pd.read_sql("""SELECT * 
                      FROM Result_data""", conn)

In [148]:
df = data.copy()

In [152]:
df['date']= pd.to_datetime(df["date"])

In [153]:
df.dtypes

level_0                              int64
index                                int64
fullVisitorId                       object
bounces                              int64
time_on_site                         int64
will_buy_on_return_visit             int64
pageviews                          float64
medium                              object
channelGrouping                     object
deviceCategory                      object
country                             object
date                        datetime64[ns]
dtype: object

In [113]:
df = df.groupby("date").agg({"pageviews":"sum",
                                      "time_on_site":"sum",
                                      "medium":"count"})
df = df.reset_index()

In [154]:
start_date_n = df_for_time.date.max()

In [155]:
end_date_n = start_date_n - timedelta(days=7)

In [156]:
start_date_n1 = end_date_n - timedelta(days=1)

In [157]:
end_date_n1 = start_date_n1 - timedelta(days=7)

In [158]:
df['last_week']= df['date'].apply(lambda x : "Dernière semaine" if end_date_n <= x <= start_date_n else  "Historique")
minimum_last = df[df['last_week']=="Dernière semaine"]["date"].min()
df_two = df[df["date"]==minimum_last]
df_two["last_week"]= "Historique"
df = pd.concat([df,df_two])

C:\Users\Simplon\AppData\Local\Temp\ipykernel_16404\1216439994.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [145]:
import plotly.express as px
import pandas as pd

#df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')

fig = px.line(df, x='date', y='pageviews', color ="last_week",title='Time Series with Rangeslider')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [97]:
df_for_time_now = df_for_time[df_for_time["date"].between(end_date_n,start_date_n)]
df_for_time_yersterday = df_for_time[df_for_time["date"].between(end_date_n1,start_date_n1)]

In [98]:
df_for_time["date"].max()

Timestamp('2023-01-31 00:00:00')

In [102]:
df_for_time_yersterday['date'].max()

Timestamp('2023-01-23 00:00:00')

In [103]:
df_for_time_yersterday['date'].min()

Timestamp('2022-12-29 00:00:00')

In [14]:
predict_data.date.max()

'2023-01-31 00:00:00'

In [ ]:

import pandas as pd
from sklearn.preprocessing import LabelEncoder
import mlflow
import sqlite3 as sql
from datetime import timedelta, datetime

conn = sql.connect('C:/Users/Simplon/Desktop/PROJET_FINAL_BEAUTY/App_Beauty_E1/Application_prod.db')

data = pd.read_sql("""SELECT * 
                      FROM Raw_data""", conn)

data["date"] = pd.to_datetime(data["date"])
today_date = "20230131"
#today_date  = datetime.today()

#today_date = datetime.strptime(date,"%Y%m%d")
last_date = today_date - timedelta(days=32)
focus_data = data.copy()[data['date'].between(last_date,today_date)]
print("---------------------------------------------------------------------------------")

df = focus_data.set_index('fullVisitorId').drop(["country",'medium','date',"index","bounces"],axis=1)
#change categorial columns

categorial = df[['channelGrouping','deviceCategory']]

for i in categorial.columns:
    df[i]= LabelEncoder().fit_transform(df[i])
    df[i].unique()
    
df = df.dropna(axis=0)

print("---------------------------------------------------------------------------------")

logged_model = 'runs:/60cfa1a830df457eb19b304db7987927/model_xgboost'
# Load model as a PyFuncModel.
loaded_model = mlflow.xgboost.load_model(logged_model)
# Predict on a Pandas DataFrame.
result = loaded_model.predict(df)
result_proba = loaded_model.predict_proba(df)[:,1]

print("---------------------------------------------------------------------------------")

data_result = focus_data.copy().dropna(axis=0)
data_result['Predict'] = result
data_result['Predict_proba']= result_proba

result_to_save = data_result[["index","fullVisitorId","date","Predict","Predict_proba"]]


print("---------------------------------------------------------------------------------")


try:
    
    check_date = pd.read_sql("SELECT * FROM Result_data",conn)
    print("Data existing wait to compare")
    
    check_date["date"] = pd.to_datetime(check_date["date"])
    if check_date[check_date["date"].between(last_date,today_date)].count()[0]!= 0 :
        print("Already Trained Data")
        if check_date[check_date["date"].between(last_date,today_date)].count()[0]<(result_to_save.count()[0]*0.85):
            pourcent_trained = round(check_date[check_date["date"].between(last_date,today_date)].count()[0] / result_to_save.count()[0] * 100,2)
            print(f"Data partialy to append check duplicate afterward.\n {pourcent_trained} % of the data already trained")
            result_to_save.to_sql("Result_data",conn, if_exists ="append",index=False)
        else: 
            print("Data trained to more than 85% data dropped")
            
    else:
        print("New Fresh Run append")
        result_to_save.to_sql("Result_data",conn, if_exists ="append",index=False)
        
        
except:
    print("first Run of infering creating table")
    
    result_to_save.to_sql("Result_data",conn, if_exists ="replace",index=False)